In [1]:
# core python 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import ipywidgets
import os
import xlrd
# tools in the simPEG Ecosystem 
import discretize  # for creating computational meshes

# linear solvers
try: 
    from pymatsolver import Pardiso as Solver  # this is a fast linear solver 
except ImportError:
    from SimPEG import SolverLU as Solver  # this will be slower

# SimPEG inversion machinery
from SimPEG import (
    Data, maps,
    data_misfit, regularization, optimization, inverse_problem, 
    inversion, directives
) 

# DC resistivity
from SimPEG.electromagnetics import resistivity as dc

# set the font size in the plots
from matplotlib import rcParams
rcParams["font.size"] = 14

In [2]:
spacing = [0,1.1,2.1,3.1,3.1,3.1,4.16,5.1,5.8,5.8,6.1,7.1,8.1,8.7,8.7,9.17,10.1,11.07,12.07,11.8,11.8,13.08,14.1,14.1,14.1,15.1,16.1,17.1,18.1,19.1,20.1,21.1,22.1,23.1,24.1,25.1,26.1,27.1,28.1,29.1,29.2,29.2,30.1,31.1,31.5,31.5,32.1,33.1,34.1,35.1,36.1,37.1,38.1,39.1,40.1,41.1,41.7,41.7,42.1,43.1,44.1,45.1,46.1,46.4,46.4,47.1,48.1,49,50,51,51.4,51.4,51.9,53,54.4]

excel_path = "D:\\惠州捷普绿点\\二维反演结果\\AM装置\\厂房内 line -- 1\\amline1无井中电极.xlsx"
excel = xlrd.open_workbook(excel_path,encoding_override="utf-8")
table = excel.sheets()[0]

col_a = table.col_values(0)
a_locations = np.zeros(len(col_a))
for i in range(0,len(col_a)):
    a_locations[i] = spacing[int(col_a[i]-1)]

col_b = table.col_values(1)
b_locations = np.zeros(len(col_b))
for i in range(0,len(col_b)):
    b_locations[i] = -120

col_m = table.col_values(2)
m_locations = []
for i in range(0,len(col_m)):
    m_locations.append(spacing[int(col_m[i]-1)])

col_n = table.col_values(3)
n_locations = []
for i in range(0,len(col_n)):
    n_locations.append(120)

col_obs = table.col_values(4)
observed_data = []
for i in range(0,len(col_obs)):
    observed_data.append(col_obs[i])

standard_deviations = []
for i in range(0,len(col_obs)):
    standard_deviations.append(0.05*observed_data[i] + 1e-3)

n_sources = len(col_obs)

dc_data_dict = {
        "a_locations": a_locations,
        "b_locations": b_locations, 
        "m_locations": m_locations,
        "n_locations": n_locations,
        "observed_data": observed_data, 
        "standard_deviations": standard_deviations,
        "n_sources": n_sources, 
}

In [3]:
underwell_list = [5,9,14,20,24,41,45,57,64,71]
underwell_depth = [-6.1,-5.1,-5.2,-5.7,-5.89,-5.8,-6,-6,-6]

inwell_list = [6,10,15,21,25,42,46,58,65,72]
inwell_depth = [-4.1,-4.1,-4.2,-3.7,-3.89,-3.8,-4,-4,-4,-4]

a_locations_z = np.zeros(len(a_locations))
for i in range(dc_data_dict["n_sources"]):
    if (col_a[i] in underwell_list):
        a_locations_z[i] = underwell_depth[underwell_list == col_a[i]]
    elif (col_a[i] in inwell_list):
        a_locations_z[i] = inwell_depth[inwell_list == col_a[i]]
    else:
        a_locations_z[i] = -0.6

m_locations_z = np.zeros(len(m_locations))
for i in range(dc_data_dict["n_sources"]):
    if (col_m[i] in underwell_list):
        m_locations_z[i] = underwell_depth[underwell_list == col_m[i]]
    elif (col_a[i] in inwell_list):
        m_locations_z[i] = inwell_depth[inwell_list == col_m[i]]
    else:
        m_locations_z[i] = -0.6

In [4]:
# initialize an empty list for each 
source_list = []

for i in range(dc_data_dict["n_sources"]):
    # receiver electrode locations in 2D 
    m_locs = np.vstack([
        dc_data_dict["m_locations"][i], 
        m_locations_z[i]
    ]).T
    n_locs = np.vstack([
        dc_data_dict["n_locations"][i],
        np.dot(-0.6, np.ones_like(dc_data_dict["n_locations"][i]))
    ]).T
    
    # construct the receiver object 
    receivers = dc.receivers.Dipole(locations_m = m_locs, locations_n = n_locs)
    
    # construct the source 
    source = dc.sources.Dipole(
        location_a=np.r_[dc_data_dict["a_locations"][i], a_locations_z[i]],
        location_b=np.r_[dc_data_dict["b_locations"][i], -0.6],
        receiver_list=[receivers]
    )

    # append the new source to the source list
    source_list.append(source)

In [5]:
survey = dc.Survey(source_list=source_list)

dc_data = Data(
    survey=survey, 
    dobs=np.hstack(dc_data_dict["observed_data"]),
    standard_deviation=np.hstack(dc_data_dict["standard_deviations"])
)

In [6]:
dx = 0.5
dz = 0.2
n_core_x = 440
n_core_z = 70
n_pad_x = 23
n_pad_z = 25
padding_factor_x = 1.3
padding_factor_z = 1.3
core_domain_x = np.array([-120,100])

In [7]:
hx = [(dx, n_pad_x, -padding_factor_x), (dx, n_core_x), (dx, n_pad_x, padding_factor_x)]
hz = [(dz, n_pad_z, -padding_factor_z), (dz, n_core_z)]
mesh = discretize.TensorMesh([hx, hz])
# origin of the mesh
mesh.x0 = np.r_[
    -mesh.hx[:n_pad_x].sum() + core_domain_x.min(),
    -mesh.hy.sum()
]
mesh

TensorMesh: 46,170 cells

                      MESH EXTENT             CELL WIDTH      FACTOR
  dir    nC        min           max         min       max      max
  ---   ---  ---------------------------  ------------------  ------
   x    486     -1,022.50      1,002.50      0.50    208.77    1.30
   y     95       -624.69          0.00      0.20    141.13    1.30

In [8]:
# define the resistivities
rho_background = 150
rho_resistive_block = 450
rho_conductive_block = 3e-4

# define the geometry of each block
xlim_resistive_block = np.r_[-90 , 60]
zlim_resistive_block = np.r_[-0.4, -0.2]

xlim_conductive_block = np.r_[-90 , 60]
zlim_conductive_block = np.r_[-0.2, 0]

In [9]:
rho = rho_background * np.ones(mesh.nC)

# resistive block
inds_resistive_block = (
    (mesh.gridCC[:, 0] >= xlim_resistive_block.min()) & (mesh.gridCC[:, 0] <= xlim_resistive_block.max()) &
    (mesh.gridCC[:, 1] >= zlim_resistive_block.min()) & (mesh.gridCC[:, 1] <= zlim_resistive_block.max())
)

rho[inds_resistive_block] = rho_resistive_block

# conductive block
inds_conductive_block = (
    (mesh.gridCC[:, 0] >= xlim_conductive_block.min()) & (mesh.gridCC[:, 0] <= xlim_conductive_block.max()) &
    (mesh.gridCC[:, 1] >= zlim_conductive_block.min()) & (mesh.gridCC[:, 1] <= zlim_conductive_block.max())
)

rho[inds_conductive_block] = rho_conductive_block

In [10]:
ind_active = ((mesh.gridCC[:, 1] > -0.2) & (mesh.gridCC[:, 0] >= -90) & (mesh.gridCC[:, 0] <= 60))
nC = int(ind_active.sum())

In [11]:
active_map = maps.InjectActiveCells(mesh, ind_active, rho[ind_active==False])
rho_map = active_map * maps.ExpMap()
starting_model = rho_conductive_block * np.ones(nC)

In [12]:
# Generate 2.5D DC problem
simulation_dc = dc.Simulation2DNodal(
    mesh, rhoMap=rho_map, solver=Solver, survey=survey, storeJ=True
)

In [13]:
def create_inverse_problem( relative_error, noise_floor, alpha_s,  alpha_x, alpha_z, mref, maxIter, maxIterCG,
):
    # set the uncertainties and define the data misfit
    dc_data.relative_error = relative_error
    dc_data.noise_floor = noise_floor
    dmisfit = data_misfit.L2DataMisfit(data=dc_data, simulation=simulation_dc)
    
    # regularization
    reg = regularization.Tikhonov(
    mesh, alpha_s=alpha_s, alpha_x=alpha_x,  alpha_y=alpha_z, ) 
    reg.mrefInSmooth = False
    reg.mref=mref*np.ones(nC)
    reg.indActive = ind_active
    # optimization 
    opt = optimization.InexactGaussNewton(maxIter=maxIter, maxIterCG=maxIterCG)
    opt.remember("xc")
    
    # return the inverse problem 
    return inverse_problem.BaseInvProblem(dmisfit, reg, opt)

In [14]:
class SaveInversionProgress(directives.InversionDirective):
    
    def initialize(self):
        # initialize an empty dictionary for storing results 
        self.inversion_results = {
            "iteration":[],
            "beta":[],
            "phi_d":[],
            "phi_m":[],
            "phi_m_small":[],
            "phi_m_smooth_x":[],
            "phi_m_smooth_z":[],
            "dpred":[],
            "model":[]
        }

    def endIter(self):
        # Save the data
        self.inversion_results["iteration"].append(self.opt.iter)
        self.inversion_results["beta"].append(self.invProb.beta)
        self.inversion_results["phi_d"].append(self.invProb.phi_d)
        self.inversion_results["phi_m"].append(self.invProb.phi_m)
        self.inversion_results["dpred"].append(self.invProb.dpred)
        self.inversion_results["model"].append(self.invProb.model)
        
        # grab the components of the regularization and evaluate them here
        # the regularization has a list of objective functions  
        # objfcts = [smallness, smoothness_x, smoothness_z]
        # and the multipliers contain the alpha values
        # multipliers = [alpha_s, alpha_x, alpha_z]
        reg = self.reg.objfcts[0] 
        phi_s = reg.objfcts[0](self.invProb.model) * reg.multipliers[0]
        phi_x = reg.objfcts[1](self.invProb.model) * reg.multipliers[1]
        phi_z = reg.objfcts[2](self.invProb.model) * reg.multipliers[2]
        
        self.inversion_results["phi_m_small"].append(phi_s)
        self.inversion_results["phi_m_smooth_x"].append(phi_x)
        self.inversion_results["phi_m_smooth_z"].append(phi_z)

In [15]:
def create_inversion( inv_prob, beta0_ratio, cool_beta, beta_cooling_factor, beta_cooling_rate, use_target, chi_factor,):
    
    # set up our directives
    beta_est = directives.BetaEstimate_ByEig(beta0_ratio=beta0_ratio)
    target = directives.TargetMisfit(chifact=chi_factor)
    save = SaveInversionProgress()
    
    directives_list = [beta_est, save]
    
    if use_target is True:
        directives_list.append(target)
    
    if cool_beta is True:
        beta_schedule = directives.BetaSchedule(coolingFactor=beta_cooling_factor, coolingRate=beta_cooling_rate)
        directives_list.append(beta_schedule)

    return inversion.BaseInversion(inv_prob, directiveList=directives_list), target, save

In [16]:
inv_prob = create_inverse_problem(
    relative_error=0.05, noise_floor=1e-3,
    alpha_s=1e-7,  alpha_x=10, alpha_z=1, mref=np.log(rho_background), 
    maxIter=20, maxIterCG=30,
)

inv, target_misfit, inversion_log = create_inversion(
    inv_prob, beta0_ratio=1e2, cool_beta=True,
    beta_cooling_factor=2, beta_cooling_rate=1,
    use_target=False, chi_factor=1 
)

phi_d_star = survey.nD / 2
target = target_misfit.chifact * phi_d_star

In [ ]:
model_recovered = inv.run(starting_model)
inversion_results = inversion_log.inversion_results

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(24,4))
rho_pre = (rho_map*model_recovered)
clim = (1e-2, 1e4)
out = mesh.plotImage(
    rho_pre, pcolorOpts={'norm':LogNorm(), 'cmap':'Spectral'}, ax=ax,
    clim = clim 
)
cb = plt.colorbar(out[0], fraction=0.05, orientation='horizontal', ax=ax, pad=0.2)
cb.set_label("Resistivity ($\Omega$m)")
ax.set_title("Inversion result")
ax.set_xlim(-100,260) 
ax.set_ylim(-10,0)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
x = np.arange(0, len(observed_data))
plt.plot(x,np.log10(dc_data.dobs),marker='o',label="dobs")
plt.plot(x,np.log10(inv_prob.dpred),marker="*",label="dpre")
plt.legend()
plt.ylabel("V")
plt.xlabel("data")

In [ ]:
rho_steel_mesh = rho_pre[ ind_active]
np.savetxt('C:\\Users\DELL\\Desktop\\rho_steelmesh.txt', rho_steel_mesh)